In [1]:
from IPython.display import clear_output

!apt-get install libmagickwand-dev
!pip install ipython-autotime
# !pip install camelot-py[cv] tabula-py
!apt-get update
!apt install ghostscript python3-tk
!pip install pdfplumber
clear_output()  

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# from tabula import read_pdf
import pandas as pd
import tqdm
from tabulate import tabulate
import pdfplumber
import glob
# import camelot
import os
import re
from functools import reduce
%load_ext autotime

time: 283 µs (started: 2022-12-06 03:19:53 +00:00)


In [3]:
sheet_id = "1ixtNKdaaFW8XNROXMWWveuCxHPErju5UUgwDw5PjM_w"
sheet_name = "Files_and_Type"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)
all_subpaths = list(df['Full Path'])

all_paths = []
for path in all_subpaths:
  t = glob.glob(f'/content/drive/MyDrive/OCR NLP Stage 2/FSSC/Documents/**/{path}', recursive=True)
  all_paths += t

time: 6.81 s (started: 2022-12-06 03:19:54 +00:00)


In [4]:
def get_trigrams(string):
    """
    Take a string and return a list of 3-grams.
    """
    s = string.strip().lower()
    return [s[i:i+3] for i in list(range(len(s) - 2))]


def string_similarity(str1, str2):
    """
    Perform bigram comparison between two strings
    and return a percentage match in decimal form.
    Input:
      str1: <str> key extracted from table
      str2: <str> keyword
    Return <float> score of similar between 2 strings
    """
    str1 = re.sub('[^A-Za-z0-9]+', '', str1.replace('\n', ''))
    str2 = re.sub('[^A-Za-z0-9]+', '', str2.replace('\n', ''))
    pairs1 = get_trigrams(str1)
    pairs2 = get_trigrams(str2)
    union = len(pairs1) + len(pairs2)
    hit_count = 0
    i = 0
    while i < len(pairs1):
      j = 0
      while j < len(pairs2):
        if pairs1[i] == pairs2[j]:
          pairs1.pop(i)
          pairs2.pop(j)
          hit_count += 1
          # s = True
          if i >= len(pairs1):break
        else:
          j += 1
      i += 1
        
    return (2.0 * hit_count) / union if union != 0 else 0

time: 3.6 ms (started: 2022-12-06 03:20:01 +00:00)


In [44]:
def contains_number(string):
    return bool(re.search(r'\d', string)) or string.lower() == 'nil'

def get_key_value(tables, keys, threshold=0.8):
  max_score = 0
  result = None
  t = ''

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          max_score = max(scores)
          result = temp[j+1]
          t = temp[j]
          print(t)

  if max_score > threshold:
    return result, max_score
  return None, None

def get_company_name(tables, keys, threshold=0.8):
  result = None
  t = ''
  results = []
  all_scores = []

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > threshold and j < len(temp)-1:
          if len(temp[j+1:]) == 2:
            results += [temp[j+2]]
            all_scores += [ max(scores) ]
          else:
            results += [temp[j+1]]
            all_scores += [ max(scores) ]
          t = temp[j]
          print(t)
  ids = list(filter(lambda x: 'n/a' not in results[x].lower(), range(len(results))))
  results = [results[i] for i in ids]
  all_scores = [all_scores[i] for i in ids]
  if len(results) > 0:
    return results[0], all_scores[0]
  return None, None

# def get_audit_recommendation(tables, rcm_keys, threshold=0.8):

  
#   rcm_tables = []
#   for i in range(len(tables)):
#     table = tables[i]
#     for _row in range(len(table)):
#       temp = np.array(list(table.iloc[_row]))
#       temp = list(temp[temp != ''])
#       if len(temp) == 0: continue
      
#       scores = list(map(lambda x: string_similarity(temp[0], x), rcm_keys))
#       if max(scores) > threshold:
#         rcm_tables += [tables[i], tables[i+1]]
#         print(f'--{temp[0]}, {max(scores)}')

#   # print(f'Found total: {len(rcm_tables)//2} keywords')
#   results = []
#   for i in range(0, len(rcm_tables), 2):
#     rcm_table = rcm_tables[i:i+2]
#     if rcm_table[0].shape[1] == rcm_table[1].shape[1]:
#       result_rcm_table = pd.concat(rcm_table, axis=0, ignore_index=True)
#       if result_rcm_table.shape[0] > 4:
#         result_rcm_table = rcm_table[0]
#     else:
#       # print(f'Table is not satisfied the shape requirements {rcm_table[0].shape}, {rcm_table[1].shape}')
#       result_rcm_table = rcm_table[0]
#     results += [result_rcm_table]
  
#   if len(results) == 0:
#     return None
#   return results[0]

def get_audit_recommendation(tables, rcm_keys, threshold=0.8):

  temp_key = 'Based on the result of this audit the audit team recommends that certification be'
  rcm_tables = []
  for i in range(len(tables)):
    table = tables[i]
    for _row in range(len(table)):
      temp = np.array(list(table.iloc[_row]))
      temp = list(temp[temp != ''])
      if len(temp) == 0: continue
      
      scores = list(map(lambda x: string_similarity(temp[0], x), rcm_keys))
      if max(scores) > threshold:
        rcm_tables += [tables[i], tables[i+1]]
        print(f'--{temp[0]}, {max(scores)}')
      
      if len(temp) == 1 and temp_key.lower() in temp[0].lower():
        return temp[0]
    
  results = []
  for i in range(0, len(rcm_tables), 2):
    rcm_table = rcm_tables[i:i+2]
    if rcm_table[0].shape[1] == rcm_table[1].shape[1]:
      result_rcm_table = pd.concat(rcm_table, axis=0, ignore_index=True)
      if result_rcm_table.shape[0] > 4:
        result_rcm_table = rcm_table[0]
    else:
      result_rcm_table = rcm_table[0]
    
    results += [result_rcm_table]
  if len(results) == 0:
    return None
  return results[0].to_string()

from dateutil import parser
def get_expiration_date_value(tables, keys, threshold=0.8):
  max_score = 0
  result = None
  t = ''
  all_dates = []

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) >= max_score and j < len(temp)-1:
          max_score = max(scores)
          if len(temp[j+1:]) < 3:
            try:
              # result = parser.parse(' '.join(temp[j+1:]), fuzzy=True).strftime("%m/%d/%Y")
              result = parser.parse(' '.join(temp[j+1:]), fuzzy=True)
              if max_score > threshold:
                all_dates += [ result ]
            except: pass
          else:
            try:
              # result = parser.parse(temp[j+1], fuzzy=True).strftime("%m/%d/%Y")
              result = parser.parse(temp[j+1], fuzzy=True)
              if max_score > threshold:
                all_dates += [ result ]
            except: pass
          t = temp[j]
  if max_score > threshold and len(all_dates) > 0:
    # return max(all_dates).strftime("%m/%d/%Y"), max_score
    return max(all_dates), max_score
  return None, None

def get_minor_value(tables, keys, threshold=0.8):
  max_score = 0
  result = None
  # t = ''

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      if len(temp) > 2:
        continue
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          max_score = max(scores)
          result = temp[j+1]
          # t = temp[j]

  if max_score > threshold and contains_number(result):
    # print(t)
    return result, max_score
  # print(f'max score: {t}, {max_score}')
  return None, None


# def get_audit_type_value(tables, keys, threshold=0.8):
#   max_score = 0
#   result = None
#   t = ''

#   for table in tables:
#     for i in range(len(table)):
#       temp = np.array(list(table.iloc[i]))
#       temp = list(temp[temp != ''])
#       for j in range(len(temp)):
#         scores = list(map(lambda x: string_similarity(temp[j], x), keys))
#         if max(scores) > max_score and j < len(temp)-1:
#           if max(scores) > threshold and len(temp) - 1 - j > 1:
#             return None, None
#           max_score = max(scores)
#           result = temp[j+1]
#           t = temp[j]

#   if max_score > threshold:
#     return result, max_score
#   return None, None

def get_audit_type_value(tables, keys, threshold=0.8):
  max_score = 0
  result = None
  t = ''

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          if max(scores) > threshold and len(temp) - 1 - j > 1:
            t = ' '.join(temp[j+1:]).split(' ') # join text from cell j+1 to the end
            if '\n' in t:
              return None, None
            else:
              max_score = max(scores)
              result = ' '.join(result[j+1:])
              # return ' '.join(result[j+1:]), max(scores)
          max_score = max(scores)
          result = temp[j+1]
          t = temp[j]

  if max_score > threshold:
    return result, max_score
  return None, None


def get_gmo_value(tables, keys, threshold=0.8):
  max_score = 0
  result = None
  t = ''

  for table in tables:
    for i in range(len(table)):
      temp = np.array(list(table.iloc[i]))
      temp = list(temp[temp != ''])
      for j in range(len(temp)):
        scores = list(map(lambda x: string_similarity(temp[j], x), keys))
        if max(scores) > max_score and j < len(temp)-1:
          max_score = max(scores)
          result = temp[j+1]
          t = temp[j]

  if max_score > threshold and contains_number(result):
    return result, max_score
  return None, None

time: 24.8 ms (started: 2022-12-06 05:26:27 +00:00)


In [7]:
# Define keyword for each field

company_name_keys = ['Company name', 'Registered legal name', 'Organisation']
address_keys = ['Address', 'Location', 'Street address, city, country', 'location/address', 'Address Line 1']
num_critical_keys = ['critical', 'critical non-conformities', 'Number of critical non-conformities identified during this audit', "Critical NCR’s"]
num_major_keys = ['major', 'major non-conformities', 'Number of major non-conformities identified during this audit', "Major NCR’s"]
num_minor_keys = ['minor', 'minor non-conformities', 'Number of minor non-conformities identified during this audit', "Minor NCR’s"]
audit_type_keys = ['Audit type']
prev_audit_type_keys = ['Previous Audit Type'] # Use in case some files use the same audit type as previous
rcm_keys = ['Audit Score/ Recommendation', 'Recommendation from this audit', 'Audit recommendation']
audit_company_keys = ['CB Name and office location', 'CB conducting audit', 'Certification Body', 'Accreditation Body(s)']
rcm_keys = ['Audit Score/ Recommendation', 'Recommendation from this audit', 'Audit recommendation']
rcm_one_row_keys = ['Based on the result of this audit the audit team recommends that certification be']
# expiration_date_keys = ['Expiration Date', 'Expiry date', 'Certificate expiry date', 'Registration']
expiration_date_keys = ['Expiration Date', 'Expiry date', 'Certificate expiry date']

time: 13.5 ms (started: 2022-12-06 03:20:22 +00:00)


In [8]:
from statistics import mode
from scipy import stats
from collections import Counter
from collections.abc import Iterable

def keep_visible_lines(mode_value):
    """
    If the object is a ``rect`` type, keep it only if the lines are visible.

    A visible line is the one having ``non_stroking_color`` as 0.
    """
    def process(obj):
      if obj['object_type'] == 'rect':
          v = tuple(obj['non_stroking_color']) if isinstance(obj['non_stroking_color'], Iterable) else obj['non_stroking_color']
          return v in mode_value
      return True
    return process

def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

def get_tables_from_file(pdf_path):
  '''
  Read all tables from pdf path
  Input 
    pdf_path: <str> path to pdf file
  Return
    List of all tables, each is a pd.DataFrame object
  '''
  tables = []
  with pdfplumber.open(pdf_path) as pdf: 
    for page_id, page in enumerate(pdf.pages):
      non_strokings = list(map(lambda x: tuple(x['non_stroking_color']) if isinstance(x['non_stroking_color'], Iterable) else x['non_stroking_color'], page.rects))
      c = Counter(non_strokings)
      if len(c) > 0 and len(page.rects) > 20:
        n = 1
        temp = sorted(c.values())[::-1]
        for i in temp[1:]:
          if temp[0] / i < 2 and i > 10: n += 1
        mode_value = list(map(lambda x: x[0], c.most_common(n)))
        # print(mode_value)
        filter_fn = keep_visible_lines(mode_value)


        page = page.filter(filter_fn)
      
      ts = {
        "vertical_strategy": "explicit",
        "horizontal_strategy": "explicit",
        "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
        "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
      } if len(page.curves + page.edges) > 0 else {
        "vertical_strategy": "lines",
        "horizontal_strategy": "lines",
      }
      page_tables = page.find_tables(ts)
      tables += page_tables
    tables = list(map(lambda table: pd.DataFrame(table.extract()).fillna(''), tables))
  return tables
      

time: 729 ms (started: 2022-12-06 03:20:36 +00:00)


In [9]:
def foo(e):
  dim = "height" if e["orientation"] == "v" else "width"
  return e[dim] <= 1

def get_tables_from_file(pdf_path):
  '''
  Read all tables from pdf path
  Input 
    pdf_path: <str> path to pdf file
  Return
    List of all tables, each is a pd.DataFrame object
  '''
  tables = []
  with pdfplumber.open(pdf_path) as pdf: 
    for page_id, page in enumerate(pdf.pages):

      temp = list(filter(lambda x: foo(x), page.edges))
      non_strokings = list(map(lambda x: tuple(x['non_stroking_color']) if isinstance(x['non_stroking_color'], Iterable) else x['non_stroking_color'], temp))

      mode_value = list(dict(Counter(non_strokings)).keys())

      if len(page.edges) > 20:
        # print(mode_value)
        filter_fn = keep_visible_lines(mode_value)
        page = page.filter(filter_fn)
      
      ts = {
        "vertical_strategy": "explicit",
        "horizontal_strategy": "explicit",
        "explicit_vertical_lines": curves_to_edges(page.curves) + page.edges,
        "explicit_horizontal_lines": curves_to_edges(page.curves) + page.edges,
      } if len(page.curves + page.edges) > 0 else {
        "vertical_strategy": "lines",
        "horizontal_strategy": "lines",
      }
      page_tables = page.find_tables(ts)
      tables += page_tables
    tables = list(map(lambda table: pd.DataFrame(table.extract()).fillna(''), tables))
  return tables
      

time: 4.46 ms (started: 2022-12-06 03:20:39 +00:00)


time: 2.32 s (started: 2022-12-04 15:18:07 +00:00)


time: 2.33 s (started: 2022-12-04 15:18:07 +00:00)


In [10]:
filenames = list(sorted(glob.glob('/content/drive/MyDrive/otraSorted_better/otraSorted/FSSC_Audit/*.pdf') ))
len(filenames)

29

time: 819 ms (started: 2022-12-06 03:20:55 +00:00)


In [42]:
fpath = filenames[2]

# fpath = all_paths[28]
print(os.path.basename(fpath))

1c4786fe-3b6b-43c6-af89-5c4f396e8973_Beef Trim N60 Addendum - 4.9.20..pdf
time: 4.16 ms (started: 2022-12-06 05:25:34 +00:00)


In [45]:
tables = get_tables_from_file(fpath)
# tables = all_datas[12]
company_name, cpn_score = get_company_name(tables, company_name_keys, threshold=0.8)
address, addr_score = get_key_value(tables, address_keys, threshold=0.8)
num_critical, critical_score = get_gmo_value(tables, num_critical_keys, threshold=0.8)
num_major, major_score = get_gmo_value(tables, num_major_keys, threshold=0.8)
num_minor, minor_score = get_minor_value(tables, num_minor_keys, threshold=0.8)

audit_type, audit_type_score = get_audit_type_value(tables, audit_type_keys, threshold=0.8)
if not audit_type:
  audit_type, audit_type_score = get_key_value(tables, prev_audit_type_keys, threshold=0.8)
audit_company, audit_company_score = get_key_value(tables, audit_company_keys, threshold=0.8)

audit_recommendation = get_audit_recommendation(tables, rcm_keys, threshold=0.8)
expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)


result = {
    'Company name': [company_name],
    'Address': [address],
    'Expiration date': [expiration_date.strftime("%m/%d/%Y") if expiration_date else None],
    'Audit company name': [audit_company],
    'Audit type': [audit_type],
    'Number critical': [num_critical],
    'Number major': [num_major],
    'Number minor': [num_minor],
    'Recommendation': [audit_recommendation]
}

result = pd.DataFrame(index=list(result.keys()), data=list(result.values()), columns=['Result'])
print(result.to_markdown(tablefmt='grid'))

Audit Result:
Auditor Email Address:
Audit ID:
+--------------------+--------------+
|                    | Result       |
+====================+==============+
| Company name       |              |
+--------------------+--------------+
| Address            |              |
+--------------------+--------------+
| Expiration date    |              |
+--------------------+--------------+
| Audit company name |              |
+--------------------+--------------+
| Audit type         | Annual audit |
+--------------------+--------------+
| Number critical    |              |
+--------------------+--------------+
| Number major       |              |
+--------------------+--------------+
| Number minor       |              |
+--------------------+--------------+
| Recommendation     |              |
+--------------------+--------------+
time: 1.42 s (started: 2022-12-06 05:26:32 +00:00)


In [ ]:
tables[5]

,0
0,(cid:36)(cid:85)(cid:72)(cid:68)(cid:86)(cid:3...


time: 9.25 ms (started: 2022-12-04 08:42:00 +00:00)


In [ ]:
# def get_expiration_date_value(tables, keys, threshold=0.8):
#   max_score = 0
#   result = None
#   t = ''
#   all_dates = []

#   for table in tables:
#     for i in range(len(table)):
#       temp = np.array(list(table.iloc[i]))
#       temp = list(temp[temp != ''])
#       for j in range(len(temp)):
#         scores = list(map(lambda x: string_similarity(temp[j], x), keys))
#         if max(scores) >= max_score and j < len(temp)-1:
#           max_score = max(scores)
#           if len(temp[j+1:]) < 3:
#             try:
#               # print(f'{temp[j]}', ' '.join(temp[j+1:]))
#               result = parser.parse(' '.join(temp[j+1:]), fuzzy=True).strftime("%m/%d/%Y")
#               # print('asdasd', ' '.join(temp[j+1:]))
#               if max_score > threshold:
#                 all_dates += [ result ]
#             except: pass
#           else:
#             try:
#               result = parser.parse(temp[j+1], fuzzy=True).strftime("%m/%d/%Y")
#               # print('asdasd', ' '.join(temp[j+1:]))
#               if max_score > threshold:
#                 all_dates += [ result ]
#             except: pass
#           t = temp[j]
#   if max_score > threshold and len(all_dates) > 0:
#     return max(all_dates), max_score
#   return None, None

time: 8.34 ms (started: 2022-12-04 07:50:43 +00:00)


In [ ]:
expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)
expiration_date

'08/20/2021'

time: 1.57 s (started: 2022-12-04 07:51:44 +00:00)


In [ ]:
tables[1]

,0,1,2,3
0,Additional \nStandard(s) (covered \nin this au...,NA,,
1,Accreditation Body(s),UKAS,,
2,Site(s) audited:,Carretera a cañadas libramientos s/n. Col. Los...,,
3,Start and end date of \naudit:,03-07-19 \n04-07-19,Last Date \nPrevious Audit,05-07-18
4,,,Certificate \nExpiry Date:,"August 20, 2021"
5,Total Man Days,3.25,Previous Audit \nRecertification \nType:,
6,Audit Time \nJustification,Hours Spent \nAccording audit plan. on Produc...,,
7,Deviation from audit \nplan and justification,No deviation,,
8,No. of FTE \nEmployees:,500,No. of Shifts: 3,
9,No. of HACCP Plans:,4,No. of \nProduction \nLines:,3


time: 13.1 ms (started: 2022-12-01 16:44:29 +00:00)


In [ ]:
tables[0]

,0,1
0,Organization profile,
1,Registeredlegalname,"IFF - International Flavors & Fragrances, Inc...."
2,Registration,
3,Location,"1620 West Crosby\nCarrollton, Texas 75006-6656..."
4,Contactperson,Lance Laplante
5,General description of audited \norganization,The facility has been part of the IFF Group si...
6,Seasonalactivities,No


time: 8.81 ms (started: 2022-12-01 13:57:45 +00:00)


### Test all paths

In [11]:
from tqdm.notebook import tqdm

time: 69.9 ms (started: 2022-12-06 03:21:12 +00:00)


In [39]:
def write_to_file(message, fname='./old_data_result2.txt'):
  with open(fname, 'a') as f:
    f.write(f'{message}\n')

time: 1.14 ms (started: 2022-12-06 05:02:26 +00:00)


In [40]:
_id = 0
for fpath in tqdm(all_paths):
# for fpath in tqdm(filenames):
  # print(f'============================================{_id}')
  tables = get_tables_from_file(fpath)
  company_name, cpn_score = get_company_name(tables, company_name_keys, threshold=0.8)
  address, addr_score = get_key_value(tables, address_keys, threshold=0.8)
  num_critical, critical_score = get_gmo_value(tables, num_critical_keys, threshold=0.8)
  num_major, major_score = get_gmo_value(tables, num_major_keys, threshold=0.8)
  num_minor, minor_score = get_minor_value(tables, num_minor_keys, threshold=0.8)

  audit_type, audit_type_score = get_audit_type_value(tables, audit_type_keys, threshold=0.8)
  if not audit_type:
    audit_type, audit_type_score = get_key_value(tables, prev_audit_type_keys, threshold=0.8)
  audit_company, audit_company_score = get_key_value(tables, audit_company_keys, threshold=0.8)

  audit_recommendation = get_audit_recommendation(tables, rcm_keys, threshold=0.8)
  # expiration_date, expr_date_score = get_key_value(tables, expiration_date_keys, threshold=0.8)
  expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)


  result = {
      'Company name': [company_name],
      'Address': [address],
      'Expiration date': [expiration_date],
      'Audit company name': [audit_company],
      'Audit type': [audit_type],
      'Number critical': [num_critical],
      'Number major': [num_major],
      'Number minor': [num_minor],
      'Recommendation': [audit_recommendation]
  }

  # for k, v in result.items():
  #   if v[0] is None:
  #     v[0] = 'Mising'

  result = pd.DataFrame(index=list(result.keys()), data=list(result.values()), columns=['Result'])
  # print(result.to_markdown(tablefmt='grid'))
  write_to_file(f'===================================={_id+3}\n{os.path.basename(fpath)}\n\n')
  write_to_file(result.to_markdown(tablefmt='grid'))
  write_to_file('\n\n\n\n')
  _id += 1

  0%|          | 0/39 [00:00<?, ?it/s]

--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit Recommendation, 1.0
--Audit Recommendation, 1.0
--Audit Recommendation, 1.0
--Audit Recommendation, 1.0
--Audit Recommendation, 1.0
--Audit Recommendation, 1.0
--Audit Recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Recommendation, 0.8275862068965517
--Recommendation, 0.8275862068965517
--Recommendation, 0.8275862068965517
--Recommendation, 0.8275862068965517
--Recommendation, 0.8275862068965517
--Recommendation, 0.8275862068965517
--Recommendation from this 
audit, 1.0
--Recommendation from this 
audit, 1.0
--The recommendation 
from this audit, 0.9433962264150944
time: 7min 9s (started: 2022-12-06 05:02:26 +00:00)


In [41]:
def write_to_file(message, fname='./production_data_result2.txt'):
  with open(fname, 'a') as f:
    f.write(f'{message}\n')

_id = 0
# for fpath in tqdm(all_paths):
for fpath in tqdm(filenames):
  # print(f'============================================{_id}')
  tables = get_tables_from_file(fpath)
  company_name, cpn_score = get_company_name(tables, company_name_keys, threshold=0.8)
  address, addr_score = get_key_value(tables, address_keys, threshold=0.8)
  num_critical, critical_score = get_gmo_value(tables, num_critical_keys, threshold=0.8)
  num_major, major_score = get_gmo_value(tables, num_major_keys, threshold=0.8)
  num_minor, minor_score = get_minor_value(tables, num_minor_keys, threshold=0.8)

  audit_type, audit_type_score = get_audit_type_value(tables, audit_type_keys, threshold=0.8)
  if not audit_type:
    audit_type, audit_type_score = get_key_value(tables, prev_audit_type_keys, threshold=0.8)
  audit_company, audit_company_score = get_key_value(tables, audit_company_keys, threshold=0.8)

  audit_recommendation = get_audit_recommendation(tables, rcm_keys, threshold=0.8)
  # expiration_date, expr_date_score = get_key_value(tables, expiration_date_keys, threshold=0.8)
  expiration_date, expr_date_score = get_expiration_date_value(tables, expiration_date_keys, threshold=0.8)


  result = {
      'Company name': [company_name],
      'Address': [address],
      'Expiration date': [expiration_date],
      'Audit company name': [audit_company],
      'Audit type': [audit_type],
      'Number critical': [num_critical],
      'Number major': [num_major],
      'Number minor': [num_minor],
      'Recommendation': [audit_recommendation]
  }

  # for k, v in result.items():
  #   if v[0] is None:
  #     v[0] = 'Mising'

  result = pd.DataFrame(index=list(result.keys()), data=list(result.values()), columns=['Result'])
  # print(result.to_markdown(tablefmt='grid'))
  write_to_file(f'===================================={_id+46}\n{os.path.basename(fpath)}\n\n')
  write_to_file(result.to_markdown(tablefmt='grid'))
  write_to_file('\n\n\n\n')
  _id += 1

  0%|          | 0/29 [00:00<?, ?it/s]

--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--The recommendation 
from this audit, 0.9433962264150944
--The recommendation 
from this audit, 0.9433962264150944
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--AUDIT RECOMMENDATION, 1.0
--Audit recommendation, 1.0
--Audit Recommendation 审核推荐, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
--Audit recommendation, 1.0
time: 5min 27s (started: 2022-12-06 05:09:36 +00:00)
